In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
PATH = 'dataset\data\\'

In [3]:
for filename in os.listdir(PATH):
    root, ext = os.path.splitext(filename)
    if ext == '.csv':
        raw_dataset = pd.read_csv(PATH + filename)

In [4]:
n = 10
raw_dataset.head(n)

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
5,x,x,x,x,o,o,b,b,o,True
6,x,x,x,x,o,b,o,o,b,True
7,x,x,x,x,o,b,o,b,o,True
8,x,x,x,x,o,b,b,o,o,True
9,x,x,x,x,b,o,o,o,b,True


In [5]:
def clean(dataset):
    size = dataset.shape[0]
    dataset = dataset[np.random.permutation(size)]
    board_x = np.where(dataset[:,:-1] == 'x', 1, np.where(dataset[:,:-1] == 'b', 0, -1))
    target_x = np.where(dataset[:,-1:], 1, 0)
    return np.concatenate((board_x, target_x), axis = 1)

In [6]:
dataset = clean(np.array(raw_dataset))
print(dataset[:n])

[[ 1 -1  0 -1  1  0 -1  1  1  1]
 [ 0 -1  0  1  1  1 -1  1 -1  1]
 [ 1 -1 -1  1 -1  1 -1  0  1  0]
 [-1  1 -1  1  1 -1 -1  1  1  1]
 [ 0  0 -1  0 -1  1 -1  1  1  0]
 [ 1  1 -1 -1 -1  0 -1  1  1  0]
 [ 0 -1  0 -1  0  0  1  1  1  1]
 [-1  0 -1  1  1  1 -1  1  0  1]
 [ 1  1 -1 -1  1  0  0  1 -1  1]
 [ 0  0  1 -1 -1 -1  1  1  0  0]]


In [7]:
size = dataset.shape[0]
x_train = (dataset[:size * 75 // 100, :-1].astype(np.float32) + 1.0 ) / 2.0
y_train = dataset[:size * 75 // 100, -1:]
x_valid = (dataset[size * 75 // 100:size * 93 // 100, :-1].astype(np.float32) + 1.0 ) / 2.0
y_valid = dataset[size * 75 // 100:size * 93 // 100, -1:]
x_test = (dataset[size * 93 // 100:,:-1].astype(np.float32) + 1.0 ) / 2.0
y_test = dataset[size * 93 // 100:, -1:]
                                            

In [8]:
np.shape(y_train)

(718, 1)

In [9]:
print(x_train)

[[1.  0.  0.5 ... 0.  1.  1. ]
 [0.5 0.  0.5 ... 0.  1.  0. ]
 [1.  0.  0.  ... 0.  0.5 1. ]
 ...
 [0.  0.5 1.  ... 1.  0.  0. ]
 [1.  0.  0.  ... 0.  1.  1. ]
 [0.  0.  0.  ... 1.  0.  1. ]]


In [10]:
K = keras.backend
kl_divergence = keras.losses.kullback_leibler_divergence

In [11]:
class KLDRegularizer(keras.regularizers.Regularizer):
    def __init__(self, weight, target = 0.1):
        self.weight = weight
        self.target = target
    def  __call__(self, inputs):
        mean_activities = K.mean(inputs, axis = 0)
        return self.weight * (
             kl_divergence(self.target, mean_activities) +
             kl_divergence(1. - self.target, 1. - mean_activities))


In [79]:
kld_reg = KLDRegularizer(weight = 0.05, target = 0.1)
encoder = keras.models.Sequential([
    keras.layers.Dense(450, activation = 'elu', input_shape = [9], kernel_initializer = 'he_normal'),
    keras.layers.Dense(300, activation = 'elu', kernel_initializer = 'he_normal'),
    keras.layers.Dense(150, activation = 'elu',  kernel_initializer = 'he_normal'),
    keras.layers.Dense(36, activation = 'sigmoid',activity_regularizer=kld_reg, kernel_initializer = 'he_normal')
    #keras.layers.ActivityRegularization(l1=1e-3)
])
decoder = keras.models.Sequential([
    keras.layers.Dense(150, activation = 'elu', input_shape = [36],  kernel_initializer = 'he_normal'),
    keras.layers.Dense(300, activation = 'elu',  kernel_initializer = 'he_normal'),
    keras.layers.Dense(450, activation = 'elu',  kernel_initializer = 'he_normal'),
    keras.layers.Dense(9, activation = 'softmax'),
])

autoencoder = keras.models.Sequential([encoder, decoder])

In [80]:
autoencoder.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_23 (Sequential)   (None, 36)                190386    
_________________________________________________________________
sequential_24 (Sequential)   (None, 9)                 190359    
Total params: 380,745
Trainable params: 380,745
Non-trainable params: 0
_________________________________________________________________


In [81]:
autoencoder.compile(loss='mean_squared_error', 
                    optimizers =  keras.optimizers.SGD(lr=0.001, momentum=0.92, nesterov=True))

In [82]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [83]:
history = autoencoder.fit(x_train[50:], x_train[50:], epochs = 120, validation_data = [x_valid, x_valid], callbacks = [early_stopping_cb])

Train on 668 samples, validate on 172 samples
Epoch 1/120
668/668 [==============================] - 1s 2ms/sample - loss: 0.3721 - val_loss: 0.3348
Epoch 2/120
668/668 [==============================] - 0s 205us/sample - loss: 0.3182 - val_loss: 0.3197
Epoch 3/120
668/668 [==============================] - 0s 173us/sample - loss: 0.3111 - val_loss: 0.3105
Epoch 4/120
668/668 [==============================] - 0s 175us/sample - loss: 0.3066 - val_loss: 0.3106
Epoch 5/120
668/668 [==============================] - 0s 170us/sample - loss: 0.3049 - val_loss: 0.3046
Epoch 6/120
668/668 [==============================] - 0s 166us/sample - loss: 0.3006 - val_loss: 0.3039
Epoch 7/120
668/668 [==============================] - 0s 170us/sample - loss: 0.3016 - val_loss: 0.3034
Epoch 8/120
668/668 [==============================] - 0s 170us/sample - loss: 0.2986 - val_loss: 0.3022
Epoch 9/120
668/668 [==============================] - 0s 169us/sample - loss: 0.2974 - val_loss: 0.3006
Epoch 10/12

In [84]:
for layer in encoder.layers[:-1]:
    layer.trainable = False

In [85]:
model = keras.models.Sequential([
    
    encoder,
    keras.layers.Dense(18, activation = 'elu', kernel_initializer = 'he_normal'),
    keras.layers.Dense(9, activation = 'elu', kernel_initializer = 'he_normal'),
    keras.layers.Dense(1, activation = 'sigmoid')
])

In [86]:
import os
import time
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
log_dir = get_run_logdir()
print(log_dir)

.\my_logs\run_2020_08_06-13_23_49


In [87]:
tensorboard_cb = keras.callbacks.TensorBoard(log_dir)
early_stopping_cb2 = keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True)

In [88]:
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [89]:
model.compile(loss='binary_crossentropy', 
            optimizers =  keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True), 
            metrics=[rounded_accuracy])

history = model.fit(x_train[:50], y_train[:50], epochs = 10, validation_data = [x_valid, y_valid], callbacks = [tensorboard_cb])

for layer in encoder.layers[:-1]:
    layer.trainable = True
    
model.compile(loss='binary_crossentropy', 
            optimizers =  keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True), 
            metrics=[rounded_accuracy])

history = model.fit(x_train[:50], y_train[:50], epochs = 50, validation_data = [x_valid, y_valid], callbacks = [tensorboard_cb,early_stopping_cb2])

Train on 50 samples, validate on 172 samples
Epoch 1/10
50/50 [==============================] - 1s 20ms/sample - loss: 0.6566 - rounded_accuracy: 0.5600 - val_loss: 0.6312 - val_rounded_accuracy: 0.6628
Epoch 2/10
50/50 [==============================] - 0s 852us/sample - loss: 0.5695 - rounded_accuracy: 0.7400 - val_loss: 0.6149 - val_rounded_accuracy: 0.6744
Epoch 3/10
50/50 [==============================] - 0s 578us/sample - loss: 0.5162 - rounded_accuracy: 0.7600 - val_loss: 0.6097 - val_rounded_accuracy: 0.6977
Epoch 4/10
50/50 [==============================] - 0s 679us/sample - loss: 0.5137 - rounded_accuracy: 0.7400 - val_loss: 0.6129 - val_rounded_accuracy: 0.6860
Epoch 5/10
50/50 [==============================] - 0s 579us/sample - loss: 0.4652 - rounded_accuracy: 0.8400 - val_loss: 0.6102 - val_rounded_accuracy: 0.6919
Epoch 6/10
50/50 [==============================] - 0s 539us/sample - loss: 0.4483 - rounded_accuracy: 0.8800 - val_loss: 0.6163 - val_rounded_accuracy: 0.6

In [90]:
model.predict_classes([[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0 ]])

array([[1]])

In [91]:
model.evaluate(x_test, y_test, verbose = False)

[0.7177933156490326, 0.7352941]

In [128]:
whole_dataset = keras.models.Sequential([
    keras.layers.Dense(450, activation = 'selu', input_shape=[9]),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(300, activation = 'selu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(150, activation = 'selu'),
    keras.layers.Dense(20, activation = 'selu'),
    keras.layers.Dense(1, activation = 'sigmoid'),
])

In [129]:
whole_dataset.compile(loss='binary_crossentropy', optimizer = 'nadam', metrics = [rounded_accuracy])

In [27]:
whole_dataset.fit(x_train, y_train, epochs = 40, validation_data = (x_valid, y_valid))

Train on 718 samples, validate on 172 samples
Epoch 1/40
718/718 [==============================] - 1s 2ms/sample - loss: 0.6575 - rounded_accuracy: 0.6880 - val_loss: 0.5765 - val_rounded_accuracy: 0.7093
Epoch 2/40
718/718 [==============================] - 0s 186us/sample - loss: 0.6012 - rounded_accuracy: 0.6671 - val_loss: 0.5304 - val_rounded_accuracy: 0.7558
Epoch 3/40
718/718 [==============================] - 0s 175us/sample - loss: 0.5767 - rounded_accuracy: 0.6992 - val_loss: 0.5137 - val_rounded_accuracy: 0.7674
Epoch 4/40
718/718 [==============================] - 0s 174us/sample - loss: 0.5864 - rounded_accuracy: 0.7075 - val_loss: 0.5451 - val_rounded_accuracy: 0.7326
Epoch 5/40
718/718 [==============================] - 0s 182us/sample - loss: 0.5637 - rounded_accuracy: 0.7145 - val_loss: 0.5178 - val_rounded_accuracy: 0.7616
Epoch 6/40
718/718 [==============================] - 0s 183us/sample - loss: 0.5322 - rounded_accuracy: 0.7228 - val_loss: 0.5905 - val_rounded_a

In [28]:
whole_dataset.evaluate(x_test, y_test, verbose = False)

[0.021706138614236432, 1.0]